# 常规赛：PALM病理性近视病灶检测与分割7月第5名方案

**比赛介绍**

	本赛题原型为ISBI2019 PALM眼科挑战赛。 近视已成为全球公共卫生负担。在近视患者中，约35%为高度近视。近视导致眼轴长度的延长，可能引起视网膜和脉络膜的病理改变。随着近视屈光度的增加，高度近视将发展为病理性近视，其特点是病理改变的形成:(1)后极，包括镶嵌型眼底、后葡萄肿、视网膜脉络膜变性等;(2)视盘，包括乳头旁萎缩、倾斜等;(3)近视性黄斑，包括漆裂、福氏斑、CNV等。病理性近视对患者造成不可逆的视力损害。因此，早期诊断和定期随访非常重要。
    
![](https://ai-studio-static-online.cdn.bcebos.com/b64226725aa74bd3a922c6ef9589a80ee0a7fe7fe66947ccace23f01a09afb5c)    

    
**赛题介绍**

	PALM病理性近视病灶检测与分割常规赛的重点是研究和发展与病理性近视诊断和患者眼底照片病变分割相关的算法。该常规赛的目标是评估和比较在一个常见的视网膜眼底图像数据集上检测病理性近视的自动算法。具体任务为：检测眼底图像是否出现视网膜萎缩病变和脱离病变，若有，需要实现病变区域的分割。
    
![](https://ai-studio-static-online.cdn.bcebos.com/70f3a304c8ec4f5f8d12f48a2bb8fc1c6ee05eaa3739475f89063873899a4ad0)
    
**数据说明**    

	数据集中每个眼底彩照上都标注了与病理性近视相关的2种典型病变：斑片状视网膜萎缩(包括乳头周围萎缩)和视网膜脱离。像素级的病灶标注首先由中山大学中山眼科中心的7名眼科医生分别手动进行，最后由另一位高级专家将7位医生的标注结果融合为一个标注金标准，并存储为BMP图像。分割图像大小与对应的眼底图像大小相同，标签如下:
   
   
   萎缩病变分割金标准：萎缩区域：0；背景：255；
   
   脱离病变分割金标准：脱离区域：0；背景：255。
    
    
**训练数据集**

	文件名称：Train

	Train里有两个文件夹，一个是fundus_images，一个是Lesion_Masks。

	* fundus_images文件夹内包含**800**张眼底彩照，分辨率为1444×1444，或2124×2056。命名形如H0001.jpg、N0001.jpg、P0001.jpg和V0001.jpg。

	* Lesion_Masks文件夹内包含两个文件夹：**Atrophy**和**Detachment**，其中，Atrophy文件夹包含fundus_images里眼底彩照的萎缩病变区域分割金标准，大小与对应的眼底彩照一致。命名前缀与对应眼底图像一致，后缀为bmp。同理，Detachment文件夹包含fundus_images里眼底彩照的脱离病变区域分割金标准，大小与对应的眼底彩照一致，命名前缀与对应眼底图像一致，后缀为bmp。**请注意，若Lesion_Masks中无某张眼底图像的病灶分割结果，说明该图像中不包含对应的病灶区域**。

**测试数据集**

	文件名称：PALM-Testing400-Images.zip

	压缩包里包含400张眼底彩照，命名形如T0001.jpg。




# 一、内容方案



* 解压数据与数据划分

* 数据标签预处理

* 利用PaddleSeg套件加速赛题开发与测试: 使用套件config中的模型yml进行快速高效的实验开发

* 实现训练流程

* 实现预测流程

* 完成提交结果 

    

    先解压需要的PadleSeg套件
  
    本项目已配置好PaddleSeg

In [ ]:
# # 解压PaddleSeg压缩包
# !unzip -oq data/data88946/PaddleSeg.zip -d /home/aistudio/
# # 修改文件名
# !mv PaddleSeg-release-v2.0 PaddleSeg

In [ ]:
# 解压数据集到PaddleSeg目录下的data文件夹
!unzip -oq data/data93538/常规赛：PALM病理性近视病灶检测与分割.zip -d PaddleSeg/data
# 移除'常规赛：PALM病理性近视病灶检测与分割'文件夹
!rm -rf PaddleSeg/data/常规赛：PALM病理性近视病灶检测与分割
!rm -rf PaddleSeg/data/__MACOSX

In [ ]:
# 查看数据集文件的树形结构
!tree -d PaddleSeg/data/常规赛：PALM病理性近视病灶检测与分割

# 二、比赛数据集情况

PALM-Testing400-Images : 测试数据集文件夹

Train : 训练数据集文件夹

* Lesion_Masks ; 标注图片

	-- Detachment 视网膜脱落标注 -- 样本较少，存在同时萎缩的样本
    
   -- Atrophy 萎缩标注
   
* fundus_image : 原始图片

> 注意没有验证数据集，这里提供一个简单的划分程序，划分比例为0.7

> utils/dataset_splited.py

# 三、划分数据集与数据预处置

当前划分比例为0.8——可在utils文件夹下的dataset_splited.py修改**train_percent**为其它值

数据预处置-可在utils文件夹下的dataset_pretrans.py中查看相关代码--实现将255转化为1，原问题变二分类问题

	注意：当前数据处理中，存在对数据进行扩充，因此当前程序运行一次之后会使得原分类数据数目增加——(扩增不宜过大，否则划分数据进行训练和验证时会出现偏差问题)


In [ ]:
# 保证路径为初始路径
%cd /home/aistudio

# 划分数据
!python utils/dataset_splited.py

# 转换标签--预处置
!python utils/dataset_pretrans.py

# 四、选择比赛模型

基线模型为:  配置略微修改的`PaddleSeg/configs/attention_unet/attention_unet_cityscapes_1024x512_80k.yml`

具体配置在
	
	-- PaddleSeg/configs/attention_unet/attention_unet_cityscapes_1024x512_80k_Deta.yml
    * 用于分割脱落情况
    
	-- PaddleSeg/configs/attention_unet/attention_unet_cityscapes_1024x512_80k_Atro.yml
    * 用于分割萎缩情况

详细数据集配置在

	-- PaddleSeg/configs/_base_/pascal_voc2012_Deta.yml
    * 用于设置脱落情况数据

	-- PaddleSeg/configs/_base_/pascal_voc2012_Atro.yml
    * 用于设置萎缩情况数据


## 1.配置_base_中对应的数据集与模型

	具体配置信息，可以在对应文件夹下查看相应`yml`文件，有相应的注释。




## 2.启动训练

### 0.下载依赖项

在平台上可以不用执行，环境支持；线下可能需要下载。

In [ ]:
# 下载依赖项，保证PaddleSeg正常运行
%cd PaddleSeg
%pwd
!pip install -r requirements.txt

### 1. 载入模型开始训练

	更换自定义的模型文件时，只需要修改对应的模型yml、数据集yml(_base_中的yml)、以及替换下方的yml即可进行训练了

	不要更改输出目录，否则后边的代码也需要修改，包括预测、后处理中的所有路径
    

In [ ]:
# 训练分割Detachment的模型，并保存到/output/Detachment  
!python train.py --c configs/attention_unet/attention_unet_cityscapes_1024x512_80k_Deta.yml \
--use_vdl \
--save_interval 200 \
--do_eval \
--seed 2200 \
--iters 4000 \
--learning_rate 0.005 \
--save_dir ./output/Detachment

In [5]:
# 训练分割Atrophy的模型，并保存到/output/Atrophy
!python train.py --c configs/attention_unet/attention_unet_cityscapes_1024x512_80k_Atro.yml \
--use_vdl \
--save_interval 200 \
--do_eval \
--seed 4396 \
--iters 20000 \
--learning_rate 0.01 \
--save_dir ./output/Atrophy

## 3.开始预测

这可以直接使用emanet进行预测，不用修改数据集yml，也不用修改相应的模型yml，注意训练权重对应即可！

	预测结果按类别分别放在`./output/result/Detachment` 和 `./output/result/Atrophy` 下

	如果使用不同的模型对不同的分割类进行讨论，注意模型yml即可

	提交结果为两种单独预测的结果

**默认使用Iou评估最好的模型训练参数--best_model**

In [7]:
%cd PaddleSeg/
# 预测Detachment
!python predict.py --config configs/attention_unet/attention_unet_cityscapes_1024x512_80k_Deta.yml \
--model_path output/Detachment/best_model/model.pdparams \
--image_path data/Detachment/Test \
--save_dir ./output/result/Detachment

# 预测Atrophy
!python predict.py --config configs/attention_unet/attention_unet_cityscapes_1024x512_80k_Atro.yml  \
--model_path output/Atrophy/best_model/model.pdparams \
--image_path data/Atrophy/Test \
--save_dir ./output/result/Atrophy

# 五、预测结果后处理

* 将类别值1换为255，进行赛题结果提交

In [8]:
%cd /home/aistudio/
!python utils/post_process.py

# 六、提交比赛结果

In [ ]:
# 复制文件到最顶层目录
%cd /home/aistudio
!cp -r PaddleSeg/output/result/ Lesion_Segmentation

# 过程移动文件--保证不包含生成的子目录
%cd Lesion_Segmentation
!cp -r Detachment/pseudo_color_prediction/. Detachment
!cp -r Atrophy/pseudo_color_prediction/. Atrophy

# 获取指定的提交目录格式
!rm -rf Detachment/added_prediction
!rm -rf Detachment/pseudo_color_prediction

!rm -rf Atrophy/added_prediction
!rm -rf Atrophy/pseudo_color_prediction

# 压缩文件
%cd /home/aistudio
!zip -r Lesion_Segmentation.zip Lesion_Segmentation
# 删除复制的文件
!rm -rf Lesion_Segmentation

In [ ]:
# # 删除zip的文件--丢失提交结果，需重新后处理生成
# !rm -rf Lesion_Segmentation.zip
# # 删除预测结果--丢失预测结果，需重新预测
# !rm -rf PaddleSeg/output/result
# # 删除output文件夹--丢失模型参数，需重新训练
# !rm -rf PaddleSeg/output
# # 删除data文件夹--数据将丢失，需要重新解压，划分，预处置
# !rm -rf PaddleSeg/data

# 七、总结与建议

	此项目根据基线版本改编，采用attention-unet的主网络结构

![](https://ai-studio-static-online.cdn.bcebos.com/fa17fa2e242f440989dadc50202b9eb70e0cb3f57a10498f821cb48fa82706ba)


* 1. 模型：本次仅采用了attention-unet，并没有实质性的改进。下次可以尝试在经典网络上，例如u-net，做些优化
* 2. 损失建议：多损失结构，不同的coef，针对赛题的特殊损失等(Dice等)
* 3. 优化器与学习率策略的调整

感谢百度PaddlePaddle提供比赛平台！！！祝愿Paddle越来越好！！！